<a href="https://colab.research.google.com/github/tfindiamooc/tfindiamooc.github.io/blob/master/colabs/CNNs_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks (CNNs)

### Import TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
import datetime, os

from tensorflow.keras import datasets, layers, models

CNNs are used extensively in Computer Vision.  They are also used for modeling temporal data.


The key idea in CNNs is to **capture local patterns in the data through a convolution operation and then to downsample the resulting output with the pooling layer**.   CNNs employ a series of convolution-pooling layers.


Let's understand these operations in the context of image classification application.  In this case CNNs take an image as an input and predicts the corresponding digit.  The image is encoded as a 3d tensor with axes corresponding to height, width and depth of an image.  


### Download and prepare the MNIST dataset

In the case of MNIST data, we are dealing with grey scale images and hence number of depth is 1 while height and width are both 28.  We will download and preprocess MNIST handwritten digit recognition dataset just as before except for a small change.   Here we reshape each image into a 3D tensor so that the input becomes a 4D tensor.  This is done to satisfy the input requirements of the CNNs.


In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

print ("Before reshaping:")
print ("=================")
print ("Number of axes in train_images: ", train_images.ndim)
print ("Number of axes in test_images: ", test_images.ndim)
print ("Shape of train_images: ", train_images.shape)
print ("Shape of test_images: ", test_images.shape)
print ()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

print ("After reshaping:")
print ("=================")
print ("Number of axes in train_images: ", train_images.ndim)
print ("Number of axes in test_images: ", test_images.ndim)
print ("Shape of train_images: ", train_images.shape)
print ("Shape of test_images: ", test_images.shape)
print ()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Before reshaping:
Number of axes in train_images:  3
Number of axes in test_images:  3
Shape of train_images:  (60000, 28, 28)
Shape of test_images:  (10000, 28, 28)

After reshaping:
Number of axes in train_images:  4
Number of axes in test_images:  4
Shape of train_images:  (60000, 28, 28, 1)
Shape of test_images:  (10000, 28, 28, 1)



We print the number of axes and shape of train and test image tensors before and after the reshaping.  You can observe that the reshaping operation converts the input 3D tensor into the 4D tensor.   An additional axes is added in each image.


### Create the convolutional base

In [0]:
model = models.Sequential()

The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument input_shape to our first layer.

In [0]:

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))


In [0]:
model.add(layers.MaxPooling2D((2, 2)))


In [0]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

In [0]:

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let display the architecture of our model so far.



In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, we can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

 Here's the complete architecture of our model.

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

As you can see, our (3, 3, 64) outputs were flattened into vectors of shape (576) before going through two Dense layers.

### Compile and train the model

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

W0725 11:30:09.988453 139963343931264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 171us/sample - loss: 0.1478 - accuracy: 0.9538
Epoch 2/5
60000/60000 [==============================] - 6s 107us/sample - loss: 0.0479 - accuracy: 0.9851
Epoch 3/5
60000/60000 [==============================] - 7s 109us/sample - loss: 0.0341 - accuracy: 0.9894
Epoch 4/5
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0270 - accuracy: 0.9915
Epoch 5/5
60000/60000 [==============================] - 6s 106us/sample - loss: 0.0215 - accuracy: 0.9934


### Evaluate the model

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

10000/10000 [==============================] - 1s 68us/sample - loss: 0.0345 - accuracy: 0.9904
0.9904


As you can see, our simple CNN has achieved a test accuracy of over 99%. 

### Classification with Feedforward Neural Networks

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
model_dnn = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_dnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model_dnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_dnn.fit(x_train, y_train, epochs=5)
model_dnn.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 93us/sample - loss: 0.3023 - accuracy: 0.9122
Epoch 2/5
60000/60000 [==============================] - 6s 93us/sample - loss: 0.1455 - accuracy: 0.9573
Epoch 3/5
60000/60000 [==============================] - 6s 93us/sample - loss: 0.1077 - accuracy: 0.9679
Epoch 4/5
60000/60000 [==============================] - 6s 92us/sample - loss: 0.0892 - accuracy: 0.9722
Epoch 5/5
10000/10000 [==============================] - 1s 57us/sample - loss: 0.0764 - accuracy: 0.9791


[0.07640231210165657, 0.9791]